In [ ]:
import os
import re
import numpy as np
import pandas as pd
from torch import nn, optim
import time

MAX_LEN = 512

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 63.5 MB/s 
     |████████████████████████████████| 84 kB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 54.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# arrange punctuation marks
def text_preprocessing(text):
    if len(text) > MAX_LEN: 
        text = text[:MAX_LEN ]
    return text

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
df_news = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bert/AAPL_finnbert.csv', sep=',', decimal=',',  index_col=0)
df_news['date'] = pd.to_datetime(df_news['date'], format="%Y/%m/%d")
df_news['date'] = df_news['date'].dt.date
df_news = df_news[~df_news.source.isin(["PR Web", "PR Newswire"])]
df_news = df_news.dropna()

df_news['title'] = df_news['title'].apply(text_preprocessing)
df_news['description'] = df_news['description'].apply(text_preprocessing)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'yiyanghkust/finbert-tone'

from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [ ]:
len(df_news)

15782

In [ ]:
i = 0
while i < len(df_news):
    print(i)
    finbert = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=3)
    tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

    if (i + 99) > len(df_news):
       df = df_news.iloc[i:,]
    else:
       df = df_news.iloc[i:i + 99,]

    headlines = list(df['description']) # list(df['title'])
    inputs = tokenizer(headlines, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]

    labels = {0:'neutral', 1:'positive', 2:'negative'}
    for idx, sent in enumerate(headlines):
        df_news.loc[df_news['description'] == sent, 'finbert_description'] = labels[np.argmax(outputs.detach().numpy()[idx])]

    i += 100

    del finbert
    del inputs
    del outputs

    time.sleep(10)

    # df_news.to_csv(f"/content/drive/My Drive/Colab Notebooks/bert/AAPL_finnbert_{i}.csv", sep=',', encoding='utf-8', header=True)

0


Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700


In [ ]:
df_news.to_csv(f"/content/drive/My Drive/Colab Notebooks/bert/AAPL_finnbert_new.csv", sep=',', encoding='utf-8', header=True)